In [1]:
import numpy as np
import mxnet as mx
from mxnet import gluon, autograd, nd
from mxnet.gluon import nn,utils 
import mxnet.ndarray as F
from tqdm import tqdm
import os

/home/skinet/py_libs/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mx.__version__

'1.1.0'

In [3]:
ctx = mx.gpu()

In [4]:
#os.environ['MXNET_ENGINE_TYPE'] = 'NaiveEngine'
#os.environ['MXNET_CUDNN_AUTOTUNE_DEFAULT'] = '1'

In [5]:
class DilatedCasualConv1d(nn.Block):
    def __init__(self,channels,dilation=1,**kwargs):
        super(DilatedCasualConv1d,self).__init__(**kwargs)
        
        self.conv = nn.Conv1D(channels,kernel_size=2,strides=1,dilation=dilation,padding=0,use_bias=False)
    
    #해당부분 확인 필요
    def init_weight_for_test(self):
        for m in self.modules():
            if isinstance(m,nn.Conv1D):
                m.weight.data()[:] = 1
    def forward(self,x):
        output = self.conv(x)
        
        return output

In [6]:
class CasualConv1d(nn.Block):
    def __init__(self,in_channels,out_channels,**kwargs):
        super(CasualConv1d,self).__init__(**kwargs)
        
        self.conv = nn.Conv1D(out_channels,kernel_size=2,strides=1,padding=1,use_bias=False)
        
    def init_weight_for_test(self):
        for m in self.modules():
            if isinstance(m,nn.Conv1D):
                m.weight.data()[:] = 1
                
    def forward(self,x):
        output = self.conv(x)
        
        return output[:,:,:-1]

In [7]:
class ResidualBlock(nn.Block):
    def __init__(self,res_channels,skip_channels,dilation,**kwargs):
        super(ResidualBlock,self).__init__(**kwargs)
        with self.name_scope():
            self.dilated = DilatedCasualConv1d(res_channels,dilation=dilation)
            self.conv_res = nn.Conv1D(res_channels,kernel_size=1)
            self.conv_skip = nn.Conv1D(res_channels,kernel_size=1)
    
    def forward(self,x,skip_size):
        output = self.dilated(x)
        
        #PixelCNN gate
        gated_tanh = F.tanh(output)
        gated_sigmoid = F.sigmoid(output)
        gated = gated_tanh * gated_sigmoid
        
        #Residual network
        output = self.conv_res(gated)
        input_cut = x[:,:,-output.shape[2]:]
        #output += input_cut
        output = output + input_cut
        
        #Skip connection
        skip = self.conv_skip(gated)
        skip = skip[:,:,-skip_size:]
        
        return output, skip
        

In [8]:
class ResidualStack(nn.Block):
    def __init__(self,layer_size,stack_size,res_channels,skip_channels,**kwargs):
        super(ResidualStack,self).__init__(**kwargs)
        
        self.layer_size = layer_size
        self.stack_size = stack_size
        
        self.res_blocks = self.stack_res_block(res_channels, skip_channels)
    
    @staticmethod
    def _residual_block(res_channels,skip_channels,dilation):
        block = ResidualBlock(res_channels,skip_channels,dilation)
        
        return block
    
    def build_dilations(self):
        dilations = []
        
        # 5 = stack[layer1, layer2, layer3, layer4, layer5]
        for s in range(0,self.stack_size):
             # 10 = layer[dilation=1, dilation=2, 4, 8, 16, 32, 64, 128, 256, 512]
            for l in range(0,self.layer_size):
                dilations.append(2 ** l)
                
        return dilations
    
    def stack_res_block(self,res_channels, skip_channels):
        
        res_blocks = []
        dilations = self.build_dilations()
        
        for dilation in dilations:
            #block = self._residual_block(res_channels, skip_channels, dilation)
            block = ResidualBlock(res_channels,skip_channels,dilation)
            block.collect_params().initialize(ctx=mx.gpu())
            res_blocks.append(block)
        return res_blocks
    
    def forward(self,x,skip_size):
        
        output = x
        skip_connections = []
        
        #print(len(self.res_blocks))
        i = 0
        for res_block in self.res_blocks:
            output, skip =  res_block(output, skip_size)
            #print(i)
            i = i+1
            #print(skip.shape)
            skip_connections.append(skip)
        #print("skip connection finished")    
        #return skip_connections
        return F.stack(*skip_connections)

In [9]:
class DenseNet(nn.Block):
    def __init__(self,channels,**kwargs):
        super(DenseNet,self).__init__(**kwargs)
        with self.name_scope():
                   
            self.conv1 = nn.Conv1D(channels,kernel_size=1)
            self.conv2 = nn.Conv1D(channels,kernel_size=1)
            
    def forward(self,x):
        output = F.relu(x)
        output = self.conv1(output)
        output = self.conv2(output)
        
        output = nd.softmax(output)
        
        return output

In [10]:
class WaveNet(nn.Block):
    def __init__(self,layer_size,stack_size,in_channels,res_channels,**kwargs):
        super(WaveNet,self).__init__(**kwargs)
        
        self.receptive_fields = self.calc_receptive_field(layer_size,stack_size)
        with self.name_scope():
            self.casual = CasualConv1d(in_channels, res_channels)
            self.res_stack = ResidualStack(layer_size,stack_size,res_channels,in_channels)
            self.densenet = DenseNet(in_channels)
    
    @staticmethod
    def calc_receptive_field(layer_size,stack_size):
        layers = [2 ** i for i in range(0,layer_size)] * stack_size
        num_receptive_fields = np.sum(layers) 
        
        return int(num_receptive_fields)
    
    def calc_output_size(self,x):
        output_size = int(x.shape[2]) - self.receptive_fields
        
        #self.check_input_size(x,output_size)
        
        return output_size

    #def check_input_size(self,x,output_size):
    
    def forward(self,x):
        
        output = F.transpose(x,axes=(0,2,1))
        output_size = self.calc_output_size(output)
        #print("casual network is executing...")
        output = self.casual(output)
        #print("skip_connections is executing...")
        #print("output :{o} output_size:{os}".format(o=output.shape,os=output_size))
        skip_connections = self.res_stack(output,output_size)
        output = skip_connections.sum(0)
        #output = np.sum(skip_connections,axis=0)
        #print("densenet is executing...")
        output = self.densenet(output)
        
        return F.transpose(output,axes=(0,2,1))
        

In [11]:
args = dict()
args['layer_size'] = 10 #10
args['stack_size'] = 5 #5
args['in_channels'] = 256
args['res_channels'] = 512
args['lr'] = 0.0002
args['num_steps'] = 10000
args['sample_size'] = 8000
args['sample_rate'] = 16000
args['data_dir'] = './data'
args['ctx'] = mx.gpu()

In [12]:
wavenet = WaveNet(args['layer_size'],args['stack_size'],args['in_channels'],args['res_channels'])

In [13]:
wavenet

WaveNet(
  (casual): CasualConv1d(
    (conv): Conv1D(None -> 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
  )
  (res_stack): ResidualStack(
  
  )
  (densenet): DenseNet(
    (conv1): Conv1D(None -> 256, kernel_size=(1,), stride=(1,))
    (conv2): Conv1D(None -> 256, kernel_size=(1,), stride=(1,))
  )
)

In [14]:
##initialize parameter
wavenet.collect_params().initialize(ctx=args['ctx'])
#set optimizer
trainer = gluon.Trainer(wavenet.collect_params(),optimizer='adam',optimizer_params={'learning_rate':args['lr'] })

/opt/venv/lib/python3.6/site-packages/mxnet/gluon/block.py:228: UserWarning: "ResidualStack.res_blocks" is a container with Blocks. Note that Blocks inside the list, tuple or dict will not be registered automatically. Make sure to register them using register_child() or switching to nn.Sequential/nn.HybridSequential instead. 
  .format(name=self.__class__.__name__ + "." + k))


In [15]:
#define loss function
loss = gluon.loss.SoftmaxCrossEntropyLoss()

## Data loading

In [16]:
from data import *

In [17]:
wavenet.receptive_fields

5115

In [18]:
args['data_dir']

'./data'

In [19]:
data_loader = DataLoader(data_dir=args['data_dir'],receptive_fields=wavenet.receptive_fields,sample_size=args['sample_size'],sample_rate=args['sample_rate'],in_channels=args['in_channels'])

In [20]:
data_loader

In [21]:
loss_for_epoch = list()

In [22]:
for e in range(args['num_steps']):
    i = 0
    for dataset in data_loader:
        for inputs, targets in tqdm(dataset):
            #print("input {i} target {t}".format(i=inputs.shape,t=targets.shape))
            inputs =inputs.copyto(mx.gpu())        
            targets = targets.copyto(mx.gpu())     
            with autograd.record():
                outputs = wavenet(inputs)
                loss_values = loss(outputs,targets)
            loss_values.backward()
            
            
            #if (i % 200 == 0):
            #    loss_for_epoch.append(loss_values.asscalar())
            #    print("{e} epoch {i} iter loss val {l}".format(e=e,i=i,l=loss_values.asscalar()))
            trainer.step(1)
            i = i + 1
    loss_for_epoch.append(loss_values.asscalar())
    print("{e} epoch {i} iter loss val {l}".format(e=e,i=i,l=loss_values.asscalar()))

2it [00:11,  5.93s/it]
1it [00:05,  5.17s/it]
2it [00:04,  2.42s/it]
5it [00:05,  1.06s/it]
2it [00:05,  2.74s/it]
1it [00:05,  5.64s/it]
1it [00:05,  5.79s/it]
2it [00:04,  2.22s/it]
1it [00:05,  5.83s/it]
2it [00:04,  2.49s/it]
2it [00:04,  2.11s/it]
4it [00:05,  1.44s/it]
5it [00:05,  1.05s/it]
1it [00:06,  6.42s/it]
4it [00:05,  1.47s/it]
1it [00:06,  6.56s/it]
1it [00:05,  5.84s/it]
4it [00:04,  1.22s/it]
1it [00:04,  4.97s/it]
5it [00:00, 11.77it/s]
2it [00:04,  2.32s/it]
2it [00:00, 15.92it/s]
2it [00:04,  2.45s/it]
5it [00:00, 12.24it/s]
2it [00:06,  3.30s/it]
1it [00:05,  5.48s/it]
3it [00:03,  1.23s/it]
1it [00:03,  3.88s/it]
4it [00:05,  1.48s/it]
1it [00:05,  5.23s/it]
2it [00:05,  2.58s/it]
2it [00:00, 13.82it/s]

0 epoch 72 iter loss val 5.544847011566162


3it [00:00, 14.64it/s]
2it [00:00, 15.09it/s]
1it [00:00, 14.33it/s]
5it [00:00,  8.42it/s]
1it [00:00,  5.78it/s]
2it [00:00, 16.81it/s]
2it [00:00, 13.02it/s]
2it [00:00, 15.50it/s]
4it [00:00, 14.15it/s]
1it [00:00, 13.65it/s]
2it [00:00, 14.45it/s]
5it [00:00, 14.09it/s]
1it [00:00, 16.76it/s]
2it [00:00, 17.30it/s]
5it [00:00, 11.26it/s]
4it [00:00, 15.12it/s]
1it [00:00, 15.54it/s]
1it [00:00, 15.60it/s]
2it [00:00, 14.77it/s]
4it [00:00, 14.39it/s]
2it [00:00,  6.27it/s]
2it [00:00, 11.95it/s]
1it [00:00, 16.89it/s]
1it [00:00, 15.58it/s]
5it [00:00, 15.24it/s]
2it [00:00, 18.22it/s]
1it [00:00, 13.55it/s]
4it [00:00, 15.03it/s]
1it [00:00, 17.15it/s]
1it [00:00, 15.77it/s]
2it [00:00, 17.12it/s]
1it [00:00, 11.39it/s]
1it [00:00, 18.68it/s]
0it [00:00, ?it/s]

1 epoch 72 iter loss val 5.470922946929932


2it [00:00, 14.06it/s]
2it [00:00, 14.09it/s]
5it [00:00, 14.45it/s]
1it [00:00, 17.42it/s]
2it [00:00, 13.87it/s]
2it [00:00, 17.20it/s]
5it [00:00, 15.51it/s]
2it [00:00, 17.22it/s]
4it [00:00, 15.28it/s]
1it [00:00, 18.51it/s]
1it [00:00, 17.84it/s]
2it [00:00, 17.20it/s]
2it [00:00, 16.70it/s]
1it [00:00, 16.01it/s]
2it [00:00, 16.97it/s]
1it [00:00, 15.83it/s]
4it [00:00, 15.05it/s]
1it [00:00, 18.49it/s]
2it [00:00, 16.84it/s]
2it [00:00, 16.96it/s]
5it [00:00, 14.77it/s]
1it [00:00, 16.00it/s]
1it [00:00, 18.01it/s]
3it [00:00, 18.53it/s]
5it [00:00, 15.85it/s]
4it [00:00, 12.59it/s]
1it [00:00, 15.72it/s]
4it [00:00, 17.27it/s]
2it [00:00, 17.06it/s]
1it [00:00,  8.56it/s]

...

102 epoch 72 iter loss val 5.447963714599609


5it [00:00, 12.82it/s]
1it [00:00, 14.77it/s]
1it [00:00, 17.47it/s]
1it [00:00, 15.54it/s]
2it [00:00, 14.80it/s]
4it [00:00, 16.05it/s]
1it [00:00, 16.74it/s]
1it [00:00, 16.92it/s]
1it [00:00, 15.56it/s]
1it [00:00, 12.20it/s]
2it [00:00, 17.02it/s]
2it [00:00, 17.01it/s]
1it [00:00, 14.66it/s]
2it [00:00, 16.26it/s]
2it [00:00, 15.95it/s]
2it [00:00, 17.63it/s]
2it [00:00, 14.41it/s]
2it [00:00, 16.65it/s]
1it [00:00, 15.75it/s]
2it [00:00, 17.97it/s]
5it [00:00, 12.21it/s]
1it [00:00, 17.38it/s]
2it [00:00, 17.28it/s]
5it [00:00, 16.30it/s]
4it [00:00, 12.07it/s]
5it [00:00, 16.93it/s]
4it [00:00, 16.63it/s]
1it [00:00, 19.38it/s]
2it [00:00, 19.50it/s]
3it [00:00, 17.00it/s]
4it [00:00, 15.41it/s]
1it [00:00,  8.51it/s]

103 epoch 72 iter loss val 5.4991607666015625


5it [00:00, 13.58it/s]
1it [00:00, 13.82it/s]
2it [00:00, 14.58it/s]
5it [00:00, 15.14it/s]
1it [00:00, 16.05it/s]
2it [00:00, 12.89it/s]
4it [00:00,  9.40it/s]
2it [00:00, 16.55it/s]
2it [00:00, 16.27it/s]
1it [00:00, 18.40it/s]
1it [00:00, 10.88it/s]
1it [00:00, 17.64it/s]
2it [00:00, 18.04it/s]
1it [00:00, 16.73it/s]
2it [00:00, 16.71it/s]
4it [00:00, 16.52it/s]
2it [00:00, 17.99it/s]
2it [00:00, 17.42it/s]
1it [00:00, 11.11it/s]
2it [00:00, 18.11it/s]
1it [00:00, 18.45it/s]
5it [00:00, 16.45it/s]
1it [00:00, 15.04it/s]
3it [00:00, 17.00it/s]
4it [00:00, 16.66it/s]
4it [00:00, 15.89it/s]
2it [00:00, 17.34it/s]
5it [00:00, 16.65it/s]
1it [00:00, 15.76it/s]
2it [00:00, 16.41it/s]
1it [00:00, 17.43it/s]


KeyboardInterrupt: 

In [23]:
filename = "wavenet.params"

In [24]:
wavenet.save_params(filename)

/opt/venv/lib/python3.6/site-packages/mxnet/gluon/block.py:228: UserWarning: "ResidualStack.res_blocks" is a container with Blocks. Note that Blocks inside the list, tuple or dict will not be registered automatically. Make sure to register them using register_child() or switching to nn.Sequential/nn.HybridSequential instead. 
  .format(name=self.__class__.__name__ + "." + k))


In [25]:
loss_for_epoch[-1]

5.518569

In [37]:
targets.shape

(1, 89)

In [63]:
inputs =inputs.copyto(mx.gpu())

In [64]:
output = wavenet(inputs)


  0%|          | 0/50 [00:00<?, ?it/s]

casual network is executing...
skip_connections is executing...
output :(1, 512, 8000) output_size:2885



100%|██████████| 50/50 [00:06<00:00,  7.72it/s]


skip connection finished
densenet is executing...


In [65]:
output.shape

(1, 2885, 256)

In [66]:
output


[[[1.77778289e-04 7.86370656e-05 1.45762417e-04 ... 7.66266836e-04
   8.07149336e-05 2.03454052e-04]
  [3.71660484e-04 7.03012774e-05 1.09471643e-04 ... 7.74058339e-04
   5.66828239e-05 1.16148069e-04]
  [1.89093218e-04 4.02802507e-05 1.96026303e-04 ... 2.61639245e-04
   6.08311675e-05 1.06477855e-04]
  ...
  [4.51185006e-05 1.23523525e-04 1.10579262e-04 ... 7.20732569e-05
   3.11169191e-04 1.32637040e-04]
  [1.83395343e-04 3.56197066e-04 2.07478806e-04 ... 1.86811871e-04
   1.28316542e-03 2.84411362e-04]
  [1.02342688e-04 1.97070374e-04 1.46624341e-04 ... 3.69880901e-04
   8.58376152e-04 1.37892275e-04]]]
<NDArray 1x2885x256 @gpu(0)>

In [67]:
targets =targets.copyto(mx.gpu())

In [69]:
b = loss(output,targets)

In [70]:
b.asscalar()

5.5452123

In [ ]:
loss(inputs)

In [31]:
104*256

26624

In [24]:
output.shape

(50, 104, 256)

In [21]:
filenames = [x for x in sorted(os.listdir( args['data_dir']))]

In [22]:
filenames

['.ipynb_checkpoints', 'miku.wav']

In [ ]:
audio = dataset.asnumpy()
audio = np.pad(audio, [[0, 0], [wavenet.receptive_fields, 0], [0, 0]], 'constant')

In [20]:
a,b = data_loader._collate_fn(dataset)

TypeError: 'NoneType' object is not iterable

In [43]:
audio = dataset.asnumpy()

In [24]:
audio = np.pad(audio, [[0, 0], [wavenet.receptive_fields, 0], [0, 0]], 'constant')

In [28]:
audio[0].shape

(2393984, 256)

In [29]:
inputs = audio[:, :args['sample_size'], :]

In [30]:
inputs.shape

(1, 100000, 256)

In [32]:
targets = audio[:, wavenet.receptive_fields:args['sample_size'], :]

In [33]:
targets.shape

(1, 94885, 256)

In [35]:
one_hot_encode(targets,2)

IndexError: arrays used as indices must be of integer (or boolean) type

In [23]:
x = nd.array([1,2])
y = nd.array([3,4])

In [24]:
c = list()
c.append(x)
c.append(y)

In [25]:
len(c)

2

In [27]:
*c

SyntaxError: can't use starred expression here (<ipython-input-27-df4d4dcd2de2>, line 1)

In [20]:
d = np.asarray(c)